In [310]:
import numpy as np
from scipy import signal
#import matplotlib.pyplot as plt
import pandas as pd
from orangebox import Parser
from collections import Counter
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [311]:
# 字段列表
fields = [
    "loopIteration",
    "time",
    "axisP[0]",
    "axisP[1]",
    "axisP[2]",
    "axisI[0]",
    "axisI[1]",
    "axisI[2]",
    "axisD[0]",
    "axisD[1]",
    "axisF[0]",
    "axisF[1]",
    "axisF[2]",
    "rcCommand[0]",
    "rcCommand[1]",
    "rcCommand[2]",
    "rcCommand[3]",
    "setpoint[0]",      # Roll
    "setpoint[1]",      # Ptich
    "setpoint[2]",      # Yaw
    "setpoint[3]",      # Throttle
    "vbatLatest",
    "amperageLatest",
    "rssi",
    "gyroADC[0]",       # Gyro Roll
    "gyroADC[1]",       # Gyro Pitch
    "gyroADC[2]",       # Gyro Yaw
    "gyroUnfilt[0]",    # Gyro Roll  Unfilted
    "gyroUnfilt[1]",    # Gyro Pitch Unfilted
    "gyroUnfilt[2]",    # Gyro Yaw   Unfilted
    "accSmooth[0]",
    "accSmooth[1]",
    "accSmooth[2]",
    "debug[0]",
    "debug[1]",
    "debug[2]",
    "debug[3]",
    "debug[4]",
    "debug[5]",
    "debug[6]",
    "debug[7]",
    "motor[0]",
    "motor[1]",
    "motor[2]",
    "motor[3]",
    "eRPM[0]",
    "eRPM[1]",
    "eRPM[2]",
    "eRPM[3]",
    "flightModeFlags",
    "stateFlags",
    "failsafePhase",
    "rxSignalReceived",
    "rxFlightChannelsValid",
    "heading[0]",
    "heading[1]",
    "heading[2]",
    "axisSum[0]",
    "axisSum[1]",
    "axisSum[2]",
    "rcCommands[0]",
    "rcCommands[1]",
    "rcCommands[2]",
    "rcCommands[3]",
    "axisError[0]",
    "axisError[1]",
    "axisError[2]"
]

# 通过字典推导式生成对应序号
field_cor_num = {field: idx for idx, field in enumerate(fields)}
print(field_cor_num)

{'loopIteration': 0, 'time': 1, 'axisP[0]': 2, 'axisP[1]': 3, 'axisP[2]': 4, 'axisI[0]': 5, 'axisI[1]': 6, 'axisI[2]': 7, 'axisD[0]': 8, 'axisD[1]': 9, 'axisF[0]': 10, 'axisF[1]': 11, 'axisF[2]': 12, 'rcCommand[0]': 13, 'rcCommand[1]': 14, 'rcCommand[2]': 15, 'rcCommand[3]': 16, 'setpoint[0]': 17, 'setpoint[1]': 18, 'setpoint[2]': 19, 'setpoint[3]': 20, 'vbatLatest': 21, 'amperageLatest': 22, 'rssi': 23, 'gyroADC[0]': 24, 'gyroADC[1]': 25, 'gyroADC[2]': 26, 'gyroUnfilt[0]': 27, 'gyroUnfilt[1]': 28, 'gyroUnfilt[2]': 29, 'accSmooth[0]': 30, 'accSmooth[1]': 31, 'accSmooth[2]': 32, 'debug[0]': 33, 'debug[1]': 34, 'debug[2]': 35, 'debug[3]': 36, 'debug[4]': 37, 'debug[5]': 38, 'debug[6]': 39, 'debug[7]': 40, 'motor[0]': 41, 'motor[1]': 42, 'motor[2]': 43, 'motor[3]': 44, 'eRPM[0]': 45, 'eRPM[1]': 46, 'eRPM[2]': 47, 'eRPM[3]': 48, 'flightModeFlags': 49, 'stateFlags': 50, 'failsafePhase': 51, 'rxSignalReceived': 52, 'rxFlightChannelsValid': 53, 'heading[0]': 54, 'heading[1]': 55, 'heading[2]'

In [ ]:
def compute_sampling_rate(df: pd.DataFrame, time_field: str) -> float:
    """
    通过时间列计算平均采样率 (Hz)。
    返回 fs。
    """
    intervals = df[time_field].diff().dropna()
    mean_interval = intervals.mean()
    fs = 1.0 / mean_interval
    print(f"平均采样间隔: {mean_interval:.6f} 秒, 采样率: {fs:.2f} Hz")
    return fs

def split_into_windows(df: pd.DataFrame, time_field: str, window_sec: float) -> list:
    """
    将 DataFrame 按照 time_field 列，以 window_sec 为长度切分成多个子 DataFrame。
    返回列表：每个元素都是一个子 DataFrame。
    """
    windows = []
    t_min = df[time_field].min()
    t_max = df[time_field].max()
    current_start = t_min

    while current_start <= t_max:
        current_end = current_start + window_sec
        win_df = df[(df[time_field] >= current_start) & (df[time_field] < current_end)]
        if not win_df.empty:
            windows.append(win_df)
            print(win_df)
        current_start = current_end

    print(f"共分割出 {len(windows)} 个非空窗口")
    return windows

def filter_windows_by_throttle(windows: list, throttle_field: str, throttle_range: tuple) -> list:
    """
    按照油门字段筛选：只保留均值在 throttle_range 之间的窗口。
    """
    valid = []
    low, high = throttle_range
    for win_df in windows:
        thr_mean = win_df[throttle_field].mean()
        if low <= thr_mean <= high:
            valid.append(win_df)
    print(f"筛选后保留 {len(valid)} 个符合油门范围的窗口")
    return valid

def process_window(win_df: pd.DataFrame,
                   input_field: str,
                   output_field: str,
                   fs: float,
                   n_fft: int,
                   displayLog: bool) -> tuple:
    """
    对单个窗口进行预处理并计算互功率谱、输入功率谱，得到 H(f)=Pxy/Pxx，
    然后提取正频率部分，返回 (freqs_pos, mag (dB), phase (deg))。
    若窗口长度 < n_fft 或标准差为零，则返回 None。
    """
    x = win_df[input_field].values
    y = win_df[output_field].values
    if len(x) < n_fft:
        print (f"Debug Info: n_fft:{n_fft}, \nwindows_len:{len(x)}")
        return None

    # 加窗
    win = np.hanning(len(x))
    x_win = x * win
    y_win = y * win

    # 去直流
    x_win = x_win - x_win.mean()
    y_win = y_win - y_win.mean()

    # 归一化
    std_x = x_win.std()
    std_y = y_win.std()
    if std_x == 0 or std_y == 0:
        return None
    x_win = x_win / std_x
    y_win = y_win / std_y

 

    # 计算互功率谱 Pxy 和输入功率谱 Pxx
    freqs, Pxy = signal.csd(y_win, x_win, fs=fs, nperseg=n_fft, window='hann')
    _, Pxx    = signal.welch(x_win, fs=fs, nperseg=n_fft, window='hann')
    H = Pxy / Pxx

    # 仅取正频率
    mask = freqs > 0
    freqs_pos = freqs[mask]
    H_pos = H[mask]

    # 幅度 (dB)
    mag = 20 * np.log10(np.abs(H_pos))
    # DC 归一化：将低于 1 Hz 的平均幅度设为 0 dB
    dc_idx = np.where(freqs_pos < 1.0)[0]
    if dc_idx.size > 0:
        dc_mag = mag[dc_idx].mean()
        mag = mag - dc_mag

    # 相位 (deg)
    phase_deg = np.unwrap(np.angle(H_pos)) * 180.0 / np.pi
    #phase_deg = np.angle(H_pos) * 180.0 / np.pi
    return freqs_pos, mag, phase_deg

def analyze_blackbox(
    file_path,
    log_index=1,               # 从1开始算
    throttle_range=(0, 9999),  # 默认不过滤
    input_field_name='',
    output_field_name='',
    window_sec=2,
    displayLog=True
):
    # ----------------- 1. 加载并解析文件，读取所有帧 -----------------
    parser = Parser.load(file_path)
    parser.set_log_index(log_index)

    data = []
    frame_gen = parser.frames()
    error_counter = Counter()

    while True:
        try:
            frame = next(frame_gen)
            data.append(frame.data)
        except StopIteration:
            break
        except Exception as e:
            error_counter[str(type(e))] += 1
            continue

    print("Skipped frames by error type:", dict(error_counter))
    if not data:
        print("未能成功解析任何帧，终止分析。")
        return

    df = pd.DataFrame(data)

    # 确保必要字段存在
    required_fields = ['time', input_field_name, output_field_name, 'setpoint[3]']
    for field in required_fields:
        if field_cor_num.get(field) not in df.columns:
            print(f"缺少必要字段：{field}")
            return

    # ----------------- 2. 时间戳转换（单位：秒） -----------------
    time_col = field_cor_num['time']
    df[time_col] = df[time_col] / 1_000_000

    # ----------------- 3. 计算采样率 -----------------
    fs = compute_sampling_rate(df, time_col)
    n_fft = fs/2

    # ----------------- 4. 分割时间窗口 -----------------
    windows = split_into_windows(df, time_col, window_sec)

    # ----------------- 5. 按油门范围过滤窗口 -----------------
    throttle_col = field_cor_num['setpoint[3]']
    valid_windows = filter_windows_by_throttle(windows, throttle_col, throttle_range)
    if not valid_windows:
        print("找不到符合油门范围的数据片段！")
        return

    # ----------------- 6. 对每个有效窗口进行频谱计算 -----------------
    all_mags = []
    all_phases = []
    freqs_pos_ref = None

    in_col  = field_cor_num[input_field_name]
    out_col = field_cor_num[output_field_name]

    for win_df in valid_windows:
        result = process_window(win_df, in_col, out_col, fs, n_fft, displayLog)
        if result is None:
            continue
        freqs_pos, mag, ph = result

        # 记录第一组频率，后续窗口须与它一致
        if freqs_pos_ref is None:
            freqs_pos_ref = freqs_pos
        all_mags.append(mag)
        all_phases.append(ph)

    if not all_mags:
        print("所有窗口在预处理后都被过滤掉了（长度不足或无方差）。")
        return

    # ----------------- 7. 计算平均增益和相位 -----------------
    avg_mag = np.mean(all_mags, axis=0)
    avg_ph  = np.mean(all_phases, axis=0)

    # ----------------- 8. 绘制 Bode 图 -----------------
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        subplot_titles=('Gain (dB)', 'Phase (deg)')
    )

    x_type = 'log' if displayLog else 'linear'
    print(f"坐标轴类型: {x_type}, 频率范围: {freqs_pos_ref.min():.2f} - {freqs_pos_ref.max():.2f} Hz")

    # 增益曲线
    fig.add_trace(
        go.Scatter(
            x=freqs_pos_ref, y=avg_mag, mode='lines',
            name='Gain (dB)',
            hovertemplate='Freq: %{x:.2f} Hz<br>Gain: %{y:.2f} dB'
        ),
        row=1, col=1
    )

    # 相位曲线
    fig.add_trace(
        go.Scatter(
            x=freqs_pos_ref, y=avg_ph, mode='lines',
            name='Phase (deg)',
            hovertemplate='Freq: %{x:.2f} Hz<br>Phase: %{y:.2f} deg'
        ),
        row=2, col=1
    )

    # 坐标轴与网格
    fig.update_xaxes(title_text="Frequency (Hz)", type=x_type, row=1, col=1)
    fig.update_xaxes(title_text="Frequency (Hz)", type=x_type, row=2, col=1)
    fig.update_yaxes(title_text="Gain (dB)", row=1, col=1)
    fig.update_yaxes(title_text="Phase (deg)", row=2, col=1)
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)

    fig.update_layout(
        height=1000, width=1000,
        title_text="Bode Plot (Interactive)",
        hovermode="x unified"
    )

    fig.show()


In [314]:
#freq_resp_unit_test
import tkinter as tk
from tkinter.filedialog import askopenfilename

bbl_path = askopenfilename()
#计算Pitch轴Setpoint与Gyro频率响应
analyze_blackbox(file_path=bbl_path, input_field_name='setpoint[2]', output_field_name='gyroADC[2]', throttle_range=(0,500))

#计算1号电机的油门信号和回传转速的频率响应
analyze_blackbox(file_path=bbl_path, input_field_name='motor[0]', output_field_name='debug[0]', throttle_range=(0,500))

Unknown event type: 15


Skipped frames by error type: {}
平均采样间隔: 0.000504 秒, 采样率: 1984.13 Hz
         0           1   2   3   4   5   6   7   8   9   ...    45    46   47  \
0         0  146.396927   4  -5  -7   0   0   0   9 -13  ...   222   309  223   
1         4  146.397575   4  -6  -8   0   0   0   9 -15  ...   235   306  223   
2         8  146.398224   4  -6  -7   0   0   0   3 -14  ...   205   302  224   
3        12  146.398873   4  -6  -7   0   0   0   3 -14  ...   203   300  225   
4        16  146.399521   4  -7  -7   0   0   0   3 -17  ...   221   298  227   
...     ...         ...  ..  ..  ..  ..  ..  ..  ..  ..  ...   ...   ...  ...   
3083  12332  148.393810   3 -16 -37  13 -22 -40  -7  16  ...  1032  1099  739   
3084  12336  148.394459  -2 -13 -36  13 -22 -41 -41  50  ...   888  1099  765   
3085  12340  148.395108  -5 -12 -36  13 -22 -41 -46  55  ...   869  1124  789   
3086  12344  148.395757  -5 -21 -35  13 -23 -42 -33   0  ...  1022  1124  767   
3087  12348  148.396405   0 -13 -34  13 

Unknown event type: 15


Skipped frames by error type: {}
平均采样间隔: 0.000504 秒, 采样率: 1984.13 Hz
         0           1   2   3   4   5   6   7   8   9   ...    45    46   47  \
0         0  146.396927   4  -5  -7   0   0   0   9 -13  ...   222   309  223   
1         4  146.397575   4  -6  -8   0   0   0   9 -15  ...   235   306  223   
2         8  146.398224   4  -6  -7   0   0   0   3 -14  ...   205   302  224   
3        12  146.398873   4  -6  -7   0   0   0   3 -14  ...   203   300  225   
4        16  146.399521   4  -7  -7   0   0   0   3 -17  ...   221   298  227   
...     ...         ...  ..  ..  ..  ..  ..  ..  ..  ..  ...   ...   ...  ...   
3083  12332  148.393810   3 -16 -37  13 -22 -40  -7  16  ...  1032  1099  739   
3084  12336  148.394459  -2 -13 -36  13 -22 -41 -41  50  ...   888  1099  765   
3085  12340  148.395108  -5 -12 -36  13 -22 -41 -46  55  ...   869  1124  789   
3086  12344  148.395757  -5 -21 -35  13 -23 -42 -33   0  ...  1022  1124  767   
3087  12348  148.396405   0 -13 -34  13 